In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import matplotlib.pyplot as plt
from json import loads

CENTER_BARCELONA = {"lat": 41.3851, "lon": 2.1734}

def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df['geometry'] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis='columns'), geometry='geometry')

In [ ]:
DATA_PATH = "./data/"

gdfs = [
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_no2_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "NO2"})
    .astype({"NO2": "category"}),
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_pm2-5_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "PM2_5"})
    .astype({"PM2_5": "category"}),
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_pm10_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "PM10"})
    .astype({"PM10": "category"}),
]

gdfs = [convert_wkt_to_geometry(gdf, "GEOM_WKT") for gdf in gdfs]
gdf = gdfs[0][["TRAM", "geometry"]]
for temp_gdf in gdfs:
    gdf = gdf.merge(temp_gdf.drop(columns=["geometry"]), on="TRAM")

gdf["NO2"] = gdf["NO2"].cat.reorder_categories(
    [
        "10-20 µg/m³",
        "20-30 µg/m³",
        "30-40 µg/m³",
        "40-50 µg/m³",
        "50-60 µg/m³",
        "60-70 µg/m³",
        ">70 µg/m³",
    ],
    ordered=True,
)

gdf["PM2_5"] = gdf["PM2_5"].cat.reorder_categories(
    ["5-10 µg/m³", "10-15 µg/m³", "15-20 µg/m³", "20-25 µg/m³", "25-30 µg/m³"],
    ordered=True,
)

gdf["PM10"] = gdf["PM10"].cat.reorder_categories(
    [
        "<=15 µg/m³",
        "15-20 µg/m³",
        "20-25 µg/m³",
        "25-30 µg/m³",
        "30-35 µg/m³",
        "35-40 µg/m³",
        "> 40 µg/m³",
    ],
    ordered=True,
)

gdf: gpd.GeoDataFrame = gdf.set_crs(epsg=25831).to_crs(epsg=4326)

In [ ]:
gdf.to_json()

In [ ]:
from shapely.ops import linemerge

# Group by NO2 and merge geometries
merged_gdf = gdf.groupby('NO2', observed=False)['geometry'].apply(func=lambda x: linemerge(x.unary_union)).reset_index()

# Convert to GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')

merged_gdf

In [ ]:
lats = []
lons = []
names = []

import shapely.geometry
import numpy as np

for feature, name in zip(merged_gdf.geometry, merged_gdf["NO2"]):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

fig = px.line_geo(lat=lats, lon=lons, hover_name=names)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_map(df, geojson=geojson, color="Bergeron",
                           locations="district_id",
                           center={"lat": 45.5517, "lon": -73.7073},
                           map_style="carto-positron", zoom=9)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.histogram(
    gdf.value_counts("NO2").reset_index(),
    x="NO2",
    y="count",
    category_orders={"NO2": gdf["NO2"].cat.categories},
    title="Distribution de la fourchette de valeurs de polluants de 2023 à Barcelone",
)
fig = fig.update_layout(
    yaxis_title="Somme des valeurs de polluants",
    xaxis_title="Fourchette de valeurs de polluants, en moyenne annuelle [µg/m3]",
)
fig.add_vline(x=1.5, line_dash="dash", line_color="orange", annotation_text=" Seuil de l'OMS", annotation_position="top right")
fig.add_vline(x=1.5, line_dash="dash", line_color="red", annotation_text=" Seuil de l'OMS", annotation_position="top right")
fig.show()

In [ ]:
px.colors.sequential.Plasma

In [ ]:
gdf["PM10"].cat.reorder_categories(['<=15 µg/m³', '15-20 µg/m³', '20-25 µg/m³', '25-30 µg/m³', '30-35 µg/m³',
       '35-40 µg/m³', '> 40 µg/m³'], ordered=True)

In [ ]:
import folium

map = folium.Map(
    location=list(CENTER_BARCELONA.values()), tiles="CartoDB Positron", zoom_start=12
)

polluant = "PM10"
# Define a color map for NO2 levels
color_map = dict(zip(gdf[polluant].cat.categories, px.colors.sequential.Plasma_r))

def style_function(feature):
    return {
        'fillColor': color_map.get(feature['properties'][polluant], 'gray'),
        'color': color_map.get(feature['properties'][polluant], 'gray'),
        'weight': 2,
        'fillOpacity': 0.6,
    }

folium.GeoJson(
    gdf.to_json(),
    name="Air Quality",
    tooltip=folium.GeoJsonTooltip(fields=["NO2", "PM2_5", "PM10"], aliases=["NO2", "PM2.5", "PM10"]),
    style_function=style_function
).add_to(map)

# Add legend
legend_html = f'''
<div style="position: fixed; 
            top: 50px; right: 50px; width: 120px; height: {35 + len(gdf[polluant].cat.categories) * 20}px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            padding: 5px 10px;
            ">
    <b>{polluant}</b><br>
    {'<br>'.join(f'<i style="background:{color_map[cat]}">&nbsp;&nbsp;&nbsp;&nbsp;</i> {cat}' for cat in gdf[polluant].cat.categories)}<br>
</div>
'''

map.get_root().html.add_child(folium.Element(legend_html))

map

In [ ]:
px.histogram(
    gdf,
    x="Rang",
    title="Histograma de la qualitat de l'aire a Barcelona",
    labels={"Rang": "Qualitat de l'aire"},
    category_orders={"Rang": gdf["Rang"].cat.categories},
)

In [ ]:
import plotly.io as pio
import dash_mantine_components as dmc

dmc.add_figure_templates(default="mantine_dark")

type(pio.templates['mantine_dark'])

pio

In [ ]:
plt.figure(figsize=(6, 10))
gdf.plot(
    ax=plt.gca(),
    column="NO2",
    legend=True
)

---

In [ ]:
df_values = pd.concat(
    [
        pd.read_csv(
            filepath_or_buffer="./data/noise_monitoring/2023/2023_1S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
        ),
        pd.read_csv(
            filepath_or_buffer="./data/noise_monitoring/2023/2023_2S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
        ),
    ]
)

df_values["date"] = pd.to_datetime(
    df_values["Any"].astype(str)
    + "-"
    + df_values["Mes"].astype(str)
    + "-"
    + df_values["Dia"].astype(str)
    + " "
    + df_values["Hora"].astype(str)
)

df_values = df_values.drop(columns=["Any", "Mes", "Dia", "Hora"])

df_insta = pd.read_csv("./data/noise_monitoring/XarxaSoroll_EquipsMonitor_Instal.csv")
df_insta = df_insta[df_insta["Id_Instal"].isin(df_values["Id_Instal"].unique())]

df_insta = df_insta[
    ["Id_Instal", "Codi_Barri", "Nom_Barri", "Codi_Districte", "Nom_Districte", "Longitud", "Latitud", "Font"]
]

df = df_values.merge(df_insta, on="Id_Instal")

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df["Longitud"], df["Latitud"]), crs="EPSG:4326"
).drop(columns=["Longitud", "Latitud"])

gdf = gdf.rename(
    columns={
        "Id_Instal": "id",
        "Nivell_LAeq_1h": "noise_level",
        "Codi_Barri": "area_code",
        "Nom_Barri": "area_name",
        "Codi_Districte": "district_code",
        "Nom_Districte": "district_name",
        "Font": "source",
    }
)

gdf = gdf.astype(
    {
        "id": "category",
        "noise_level": "float32",
        "area_code": "category",
        "area_name": "category",
        "district_code": "category",
        "district_name": "category",
        "source": "category",
    }
)

gdf["source"] = gdf["source"].cat.rename_categories(
    {
        "ACTIVITATS / INFRASTRUCTURES ESPORTIVES": "ACTIVITÉS SPORTIVES / INFRASTRUCTURES",
        "ANIMALS": "ANIMAUX",
        "NETEJA": "NETTOYAGE",
        "OBRES": "TRAVAUX",
        "OCI": "LOISIRS",
        "PATIS D'ESCOLA": "COURS D'ÉCOLE",
        "TRÀNSIT": "TRAFIC",
        "XARXA DE TRANSPORT PÚBLIC": "RÉSEAU DE TRANSPORT PUBLIC",
        "ZONES PEATONALS": "ZONES PIÉTONS",
    }
)

In [ ]:
gdf_map = gdf.drop_duplicates("id")
fig = px.scatter_mapbox(
    gdf_map,
    lat=gdf_map.geometry.y,
    lon=gdf_map.geometry.x,
    color="source",
    hover_name="source",
    hover_data=["noise_level", "district_name", "area_name"],
    title="Carte des capteurs de bruit à Barcelone en 2023",
    center=CENTER_BARCELONA,
    zoom=12,
    height=600,
)

fig.update_layout(
    mapbox_style="carto-positron",
    legend_title_text="Type de bruit",
    title={"y": 0.98, "x": 0.01},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()

In [ ]:
fig = px.line(
    gdf.groupby("date")['noise_level'].mean().reset_index(),
    x="date",
    y="noise_level",
    title="Niveaux de bruit moyen à Barcelone en 2023",
)

fig.update_layout(xaxis_rangeslider_visible=True)

fig.show()

In [ ]:
import plotly.express as px

fig = px.histogram(
    gdf.groupby(["date", "source"], observed=False)['noise_level'].mean().reset_index(),
    x="noise_level",
    marginal="box",
    color="source",
    title="Distribution des niveaux de bruit moyen par source à Barcelone en 2023",
)

fig = fig.update_layout(
    xaxis_title="Niveau sonore moyen [dB]",
)

# Ajouter les lignes verticales sans annotation dans add_vline
fig.add_vline(x=50, line_dash="dash", line_color="#0097A7")
fig.add_vline(x=60, line_dash="dash", line_color="#74BE41")
fig.add_vline(x=70, line_dash="dash", line_color="#D5DE20")
fig.add_vline(x=80, line_dash="dash", line_color="#EF4429")

# Ajouter des annotations manuelles en dehors du graphique principal
annotations = [
    dict(x=50, y=1.1, xref="x", yref="paper", text="Faible", showarrow=False, font=dict(color="#0097A7")),
    dict(x=60, y=1.1, xref="x", yref="paper", text="Modéré à silencieux", showarrow=False, font=dict(color="#74BE41")),
    dict(x=70, y=1.1, xref="x", yref="paper", text="Bruyant", showarrow=False, font=dict(color="#D5DE20")),
    dict(x=80, y=1.1, xref="x", yref="paper", text="Très Bruyant", showarrow=False, font=dict(color="#EF4429")),
]

fig.update_layout(annotations=annotations)

fig.show()


In [ ]:
fig = px.sunburst(
    gdf.drop_duplicates("id"),
    path=["district_name", "source"],
    title="Répartition des sources de bruit par district à Barcelone en 2023",
)
fig.update_traces(textinfo="label+percent entry")
fig.show()

In [ ]:
fig = px.pie(
    gdf.drop_duplicates("id").value_counts("source").reset_index(),
    names="source",
    values="count",
    title="Répartition des sources de bruit à Barcelone en 2023",
)
fig.update_traces(textposition='inside', textinfo='percent+label', hole=.3)
fig.update_layout(showlegend=False, margin={"r": 0, "t": 0, "l": 0, "b": 0},)
fig.show()

---

In [ ]:
df = pd.read_csv("./data/district_zone/BarcelonaCiutat_Districtes.csv")

gdf = convert_wkt_to_geometry(df, 'geometria_wgs84')

In [ ]:
fig = px.choropleth_map(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="nom_districte",
    center=CENTER_BARCELONA,
    title="Map of Districts in Barcelona"
)

fig.show()

---

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

DATA_PATH = "./data/"

# --- COMMON FUNCTIONS ---


def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df["geometry"] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis="columns"), geometry="geometry")


# --- NOISE DATA ---


def load_noise_data() -> gpd.GeoDataFrame:
    df_values = pd.concat(
        [
            pd.read_csv(
                filepath_or_buffer="./data/noise_monitoring/2023/2023_1S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
            ),
            pd.read_csv(
                filepath_or_buffer="./data/noise_monitoring/2023/2023_2S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
            ),
        ]
    )

    df_values["date"] = pd.to_datetime(
        df_values["Any"].astype(str)
        + "-"
        + df_values["Mes"].astype(str)
        + "-"
        + df_values["Dia"].astype(str)
        + " "
        + df_values["Hora"].astype(str)
    )

    df_values = df_values.drop(columns=["Any", "Mes", "Dia", "Hora"])

    df_insta = pd.read_csv(
        "./data/noise_monitoring/XarxaSoroll_EquipsMonitor_Instal.csv"
    )
    df_insta = df_insta[df_insta["Id_Instal"].isin(df_values["Id_Instal"].unique())]

    df_insta = df_insta[
        [
            "Id_Instal",
            "Codi_Barri",
            "Nom_Barri",
            "Codi_Districte",
            "Nom_Districte",
            "Longitud",
            "Latitud",
            "Font",
        ]
    ]

    df = df_values.merge(df_insta, on="Id_Instal")

    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df["Longitud"], df["Latitud"]), crs="EPSG:4326"
    ).drop(columns=["Longitud", "Latitud"])

    gdf = gdf.rename(
        columns={
            "Id_Instal": "id",
            "Nivell_LAeq_1h": "noise_level",
            "Codi_Barri": "area_code",
            "Nom_Barri": "area_name",
            "Codi_Districte": "district_code",
            "Nom_Districte": "district_name",
            "Font": "source",
        }
    )

    gdf = gdf.astype(
        {
            "id": "category",
            "noise_level": "float32",
            "area_code": "category",
            "area_name": "category",
            "district_code": "category",
            "district_name": "category",
            "source": "category",
        }
    )

    gdf["source"] = gdf["source"].cat.rename_categories(
        {
            "ACTIVITATS / INFRASTRUCTURES ESPORTIVES": "ACTIVITÉS SPORTIVES / INFRASTRUCTURES",
            "ANIMALS": "ANIMAUX",
            "NETEJA": "NETTOYAGE",
            "OBRES": "TRAVAUX",
            "OCI": "LOISIRS",
            "PATIS D'ESCOLA": "COURS D'ÉCOLE",
            "TRÀNSIT": "TRAFIC",
            "XARXA DE TRANSPORT PÚBLIC": "RÉSEAU DE TRANSPORT PUBLIC",
            "ZONES PEATONALS": "ZONES PIÉTONS",
        }
    )

    return gdf


gdf_noise = load_noise_data()


# --- AIR DATA ---


def load_air_data() -> gpd.GeoDataFrame:
    gdfs = [
        pd.read_csv(
            DATA_PATH + "air_quality/2023/2023_tramer_no2_mapa_qualitat_aire_bcn.csv"
        )
        .rename(columns={"Rang": "NO2"})
        .astype({"NO2": "category"}),
        pd.read_csv(
            DATA_PATH + "air_quality/2023/2023_tramer_pm2-5_mapa_qualitat_aire_bcn.csv"
        )
        .rename(columns={"Rang": "PM2_5"})
        .astype({"PM2_5": "category"}),
        pd.read_csv(
            DATA_PATH + "air_quality/2023/2023_tramer_pm10_mapa_qualitat_aire_bcn.csv"
        )
        .rename(columns={"Rang": "PM10"})
        .astype({"PM10": "category"}),
    ]

    gdfs = [convert_wkt_to_geometry(gdf, "GEOM_WKT") for gdf in gdfs]
    gdf: gpd.GeoDataFrame = gdfs[0][["TRAM", "geometry"]]
    for temp_gdf in gdfs:
        gdf = gdf.merge(temp_gdf.drop(columns=["geometry"]), on="TRAM")

    gdf["NO2"] = gdf["NO2"].cat.reorder_categories(
        [
            "10-20 µg/m³",
            "20-30 µg/m³",
            "30-40 µg/m³",
            "40-50 µg/m³",
            "50-60 µg/m³",
            "60-70 µg/m³",
            ">70 µg/m³",
        ],
        ordered=True,
    )

    gdf["PM2_5"] = gdf["PM2_5"].cat.reorder_categories(
        ["5-10 µg/m³", "10-15 µg/m³", "15-20 µg/m³", "20-25 µg/m³", "25-30 µg/m³"],
        ordered=True,
    )

    gdf["PM10"] = gdf["PM10"].cat.reorder_categories(
        [
            "<=15 µg/m³",
            "15-20 µg/m³",
            "20-25 µg/m³",
            "25-30 µg/m³",
            "30-35 µg/m³",
            "35-40 µg/m³",
            "> 40 µg/m³",
        ],
        ordered=True,
    )

    return gdf.set_crs(epsg=25831).to_crs(epsg=4326)


gdf_air = load_air_data()
gdf_air_json = gdf_air.to_json()

In [ ]:
park_trees_df = pd.read_csv(
    DATA_PATH + "trees/street_trees/2023_4T_OD_Arbrat_Viari_BCN.csv"
)
street_trees_df = pd.read_csv(
    DATA_PATH + "trees/street_trees/2023_4T_OD_Arbrat_Viari_BCN.csv"
)
zone_trees_df = pd.read_csv(
    DATA_PATH + "trees/zone_trees/2023_4T_OD_Arbrat_Zona_BCN.csv"
)

all_trees_df = pd.concat(
    [park_trees_df, street_trees_df, zone_trees_df]
)  # Concatenate the three DataFrames

# Get the number of trees per district
trees_per_district = (
    all_trees_df[["codi_districte", "nom_districte"]].value_counts(sort=1).reset_index()
)
trees_per_district.columns = ["Code", "District", "Number of Trees"]
trees_per_district.loc[len(trees_per_district.index)] = [
    "0",
    "TOTAL",
    all_trees_df.shape[0],
]

# Create a DataFrame with the areas of each district
district_areas_df = pd.DataFrame(
    {
        "District": [
            "CIUTAT VELLA",
            "EIXAMPLE",
            "SANTS - MONTJUÏC",
            "LES CORTS",
            "SARRIÀ - SANT GERVASI",
            "GRÀCIA",
            "HORTA - GUINARDÓ",
            "NOU BARRIS",
            "SANT ANDREU",
            "SANT MARTÍ",
            "TOTAL",
        ],
        "Area": [
            4.11,
            7.46,
            22.68,
            6.02,
            19.91,
            4.19,
            11.96,
            8.05,
            6.59,
            10.39,
            101.36,
        ],
    }
)

# Calculate the number of trees per km²
trees_per_district = trees_per_district.merge(district_areas_df, on="District")
trees_per_district["Trees per km²"] = (
    trees_per_district["Number of Trees"] / trees_per_district["Area"]
)

district_df = pd.read_csv(DATA_PATH + "district_zone/BarcelonaCiutat_Districtes.csv")

# Convert the WKT geometries to Shapely geometries
district_df = convert_wkt_to_geometry(district_df, "geometria_wgs84").set_crs(epsg=4326)

# Create a GeoDataFrame with the number of trees per district
district_df = district_df.merge(
    trees_per_district, left_on="Codi_Districte", right_on="Code"
).drop(columns=["geometria_etrs89", "District"])

gdf_trees = district_df.rename(
    columns={
        "Codi_Districte": "district_code",
        "nom_districte": "district_name",
    }
)[["district_code", "district_name", "Number of Trees", "Trees per km²", "geometry"]]

gdf_trees = gdf_trees.astype(
        {
            "district_code": "category",
            "district_name": "category",
            "Number of Trees": "int32",
            "Trees per km²": "float32",
        }
    )

In [ ]:
gdf_trees

---